In [1]:
import os

os.chdir("..")
print(os.getcwd())

/workspaces/mastering-python/logging


In [2]:
from dotenv import load_dotenv

environment = "dev"
print(
    "successfully loaded env file: ",
    load_dotenv(f"./env/.env.{environment}", override=True),
)

successfully loaded env file:  True


**README**

logging telemetry to azure: you need both a log analytics workspace (LAW) and an application insights (AI) resource. The LAW is the central log database for all Azure Monitor features. The AI resource ingests the logs and makes sure it captures all the opentelemetry information and serializes them.

you can query those f.ex. like this:

```KQL
traces
| where isnotempty(customDimensions["user_id"])
| project 
    timestamp,
    severity = severityLevel,
    msg = message,
    user_id = customDimensions["user_id"],
    collection = customDimensions["collection"],
    changes = customDimensions["changes"]
```

# Telemetry

## Basics

NOTE: sometimes it takes a while for the logs to show up in azure - try query with different time ranges. Setting `enable_live_metrics=True` can help though.

following this: [docs](https://learn.microsoft.com/en-us/azure/azure-monitor/app/opentelemetry-enable?tabs=python#enable-azure-monitor-opentelemetry-for-net-nodejs-python-and-java-applications)

In [3]:
import logging

from azure.monitor.opentelemetry import configure_azure_monitor

logging.basicConfig(
    format="{levelname} {name}: {message}",
    style="{",
)

configure_azure_monitor(
    logger_name="app",
    connection_string=os.environ["APPLICATIONINSIGHTS_CONNECTION_STRING"],
    enable_live_metrics=True,
    disable_offline_storage=True,
)
logger = logging.getLogger("app")
logger.setLevel(logging.INFO)

In [ ]:
logger.error(
    "CriticalUpload",
    extra={
        "user_id": "user_id_1234",
        "file_count": 3,
        "success": False,
        "duration": 2.3,
    },
)
logger.info(
    "FileUploaded",
    extra={
        "user_id": "user_id_1234",
        "file_count": 2,
        "success": True,
        "duration": 2.2,
    },
)

ERROR app: this is critical
INFO app: FileUploaded
